参考：[Deep Learning based image Super-Resolution to enhance photos](http://cv-tricks.com/deep-learning-2/image-super-resolution-to-enhance-photos/)

# 什么是图像超分辨率？

**图像超分辨率**是一种软件技术，它可以让我们用现有的硬件增强图像的空间分辨率。

- **低分辨率（LR）图像**：图像中的像素密度很小，因此它提供的细节很少。
- **高分辨率（HR）图像**：图像中的像素密度很大，因此它提供了大量细节。

用于通过恢复高频细节从一个或多个低分辨率图像重建高分辨率图像的技术被称为“**超分辨率**”。

当我们简单地在 OpenCV 或 Scipy 中调整图像大小时，使用诸如“插值”的传统方法， 其基于附近的像素值来近似新像素的值，这在视觉质量方面有很多不足之处，如细节（例如，锐利的）边缘）通常不被保留。

在这里，我们使用深度学习来学习使用生成对抗网络预测这些值。训练模型可用于生成具有来自低分辨率图像的细节的高分辨率图像。

# 了解基于深度学习的超级分辨率：

好吧，让我们考虑如何构建卷积神经网络来训练模型，将空间大小增加 $4$ 倍。我们已经知道卷积运算总是减小输入的大小。因此，我们将不得不使用反卷积或分步跨越卷积或类似层（子像素卷积层），使输出图像大小为输入大小的 $4$ 倍。训练数据很简单：我们可以从互联网上收集大量高分辨率（HR）图像，然后将它们缩小 $4$ 倍，这将是我们的低分辨率（LR）图像。我们将低分辨率图像（比方说大小为 $20\times 20$）提供给网络并训练它以生成高分辨率图像（$80\times 80$）。网络的目的是减小所生成图像的像素与地面实况图像之间的均方误差（MSE）。

$$
MSE = \displaystyle{\sum_{i=1}^M \sum_{j=1}^N} ||f(i,j) - g(i,j)||^2 / M \cdotp N
$$

- $f$ 表示原始图像的矩阵
- $g$ 表示重建的高分辨率图像的矩阵
- $M$ 表示图像的像素行数，$i$ 表示该行的索引
- $N$ 表示图像的像素列数，$j$ 表示该列的索引

假设，如果我们将此错误归零，则意味着我们的网络现在能够生成高质量的高分辨率图像。因此，我们定义了质量指标：

# 峰值信噪比(PSNR)

它测量生成的高分辨率图像与原始图像（自然高分辨率图像）之间的偏差。它可以定义为输入图像的最大像素值（峰值信号）之间的比率（例如，如果输入图像是 $8$ 位无符号整数数据类型，那么该值将等于 $255$）到 MSE（Mean Squared）在重建图像的像素值与以对数标度表示的原始图像之间的误差。

$$
PSNR = 10 \log ({maxvalue^2/MSE})
$$

这里
- $maxvalue$ 表示输入图像中存在的最大像素值（原始目标图像）

PSNR 越高，重建图像的质量越好，因为它试图使图像之间的 MSE 相对于输入图像的最大像素值最小化。 

不幸的是，单独最大化 PSNR 对我们来说并不完全有效，因为生成的图像可能过于平滑，看起来并不真实。因此，我们使用深度神经网络（比如 Vgg19 或 Alexnet）作为特征提取器，并将生成的图像和地面实况图像之间的特征图差异作为该网络的损失。这种损失称为**内容丢失**。

# 判别网络和对抗性损失：

如前所述，PSNR 不是预测图像是否真实的完美度量标准。如果我们能够判断生成的图像的质量并拒绝那些不现实的图像，那就更好了。这就是为什么我们使用另一个网络来预测生成的图像有多好/真实。该网络被称为鉴别器网络，因为它试图预测由发生器产生的图像是否真实。

![overly_smooth_patches.jpg](http://cv-tricks.com/wp-content/uploads/2018/07/overly_smooth_patches.png)

因此，实际上我们使用生成的具有两个网络的对抗网络，生成器将低分辨率图像作为输入并生成高分辨率图像作为输出。鉴别器决定生成图像的感知真实程度。鉴别器还向生成器添加称为对抗性损失的反馈，其预测生成的图像是否真实的概率。生成器使用它作为改进的提示。这是完整的培训模式：
![](http://cv-tricks.com/wp-content/uploads/2018/07/Super_Resolution_training-2.jpg)

开始时，发生器会产生质量非常差的图像，但随着训练的继续，它会开始产生逼真的图像。到训练完成时，鉴别器不能很好地区分原始高分辨率图像和由发生器创建的图像。

# 好莱坞/ CSI版的超级分辨率是否可能?

您可以理解，我们无法创建原始图像中不存在的任何信息。我们创建的新像素是基于训练的非常好的猜测。然而，它确实改善了图像，使人类更容易想象。因此，如果我们有一个低分辨率的图像只缺少一些像素，这使得人类难以解释，但信息存在，那么超分辨率肯定会有所帮助。然而，在许多情况下，似乎原始的低分辨率图像对于人类来说非常难以可视化和解释，这使得它感觉像魔术一样。

总的来说，超分辨率是一种非常酷的深度学习应用。现在可以构建非常酷的图像增强软件，深度学习自动将超分辨率应用于图像。毫无疑问，正如许多深度学习模型的情况一样，训练特定于领域的模型（如面孔或车牌）非常有效，我们已经看到超级分辨率真的有奇迹！